In [1]:
import pyodbc
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [3]:
server = "HE170351" # database server name
database = "LAB_03"  # database name

connection_str = ("Driver={SQL Server Native Client 11.0};"
                 "Server=" + server + ";"
                 "Database=" + database + ";"
                 "Trusted_Connection=yes;")

try:
    connection = pyodbc.connect(connection_str)
    print("Connected to Server: " + server + "\nDatabase: "+ database)
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print("Unable to connect: " + sqlstate)

Connected to Server: HE170351
Database: LAB_03


### Count the number of elementary school, middle school and high school in a query.

In [12]:
query = "SELECT SUM(CASE WHEN Elementary_Middle_or_High_School = 'ES' THEN 1 ELSE 0 END) AS Elementary_School, SUM(CASE WHEN Elementary_Middle_or_High_School = 'MS' THEN 1 ELSE 0 END) AS Middle_School,SUM(CASE WHEN Elementary_Middle_or_High_School = 'HS' THEN 1 ELSE 0 END) AS High_School FROM dbo.Chicago"
data = pd.read_sql(query, connection)
data

,Elementary_School,Middle_School,High_School
0,462,11,93


### Find the school that the lowest average student attendance.

In [13]:
query = "SELECT TOP 1 Name_of_School,Average_Student_Attendance FROM dbo.Chicago WHERE Average_Student_Attendance IS NOT NULL ORDER BY Average_Student_Attendance"
data = pd.read_sql(query, connection)
data

,Name_of_School,Average_Student_Attendance
0,Richard T Crane Technical Preparatory High School,57.900002


### Find the average environment score.

In [14]:
query = "SELECT AVG(Environment_Score) AS Average_Environment_Score FROM dbo.Chicago"
data = pd.read_sql(query, connection)
data

,Average_Environment_Score
0,47


### Find the number of Elementary School in community area WEST TOWN.

In [15]:
query = "SELECT COUNT(*) AS Number_Of_Elementary_West_Town FROM dbo.Chicago WHERE Community_Area_Name = 'WEST TOWN' AND Elementary_Middle_or_High_School = 'ES'"
data = pd.read_sql(query, connection)
data

,Number_Of_Elementary_West_Town
0,17


### Count all the High School that has name start by letter ‘C’.

In [16]:
query = "SELECT COUNT(*) AS Number_Of_High_School_Start_By_C FROM dbo.Chicago WHERE Name_of_School = 'C%' AND Elementary_Middle_or_High_School = 'HS'"
data = pd.read_sql(query, connection)
data

,Number_Of_High_School_Start_By_C
0,0


### Find the top 10 Middle School has lowest Safety Score.

In [17]:
query = "SELECT TOP 10 Name_of_School, Safety_Score FROM dbo.Chicago WHERE Elementary_Middle_or_High_School = 'MS' ORDER BY Safety_Score"
data = pd.read_sql(query, connection)
data

,Name_of_School,Safety_Score
0,Northwest Middle School,34
1,Irene C. Hernandez Middle School for the Advan...,34
2,Ames Middle School,38
3,Thurgood Marshall Middle School,39
4,Pershing West Middle School,46
5,Francisco I Madero Middle School,49
6,Williams Preparatory Academy Middle School,49
7,Logandale Middle School,56
8,Miriam G Canter Middle School,57
9,Evergreen Academy Middle School,60


### Find all school has missing value in Leader Score column.

In [18]:
query = "SELECT Name_of_School FROM dbo.Chicago WHERE Leaders_Score IS NULL"
data = pd.read_sql(query, connection)
data

,Name_of_School
0,Austin Business and Entrepreneurship Academy H...
1,William Jones College Preparatory High School
2,Walter Payton College Preparatory High School
3,North-Grand High School
4,Neal F Simeon Career Academy High School
...,...
266,South Shore Fine Arts Academy
267,Dr Jorge Prieto Math and Science
268,Skinner North
269,Mason High School


### Update all missing value Leader Score column by mean of available data.

In [30]:
# Tính toán giá trị trung bình của Leader Score
query = "SELECT AVG(Leaders_Score) AS mean_score FROM Chicago"
data = pd.read_sql(query, connection)
data

,mean_score
0,49


In [35]:
# Cập nhật giá trị thiếu bằng giá trị trung bình
update_query = f"UPDATE Chicago SET Leaders_Score = 49 WHERE Leaders_Score IS NULL"
cursor = connection.cursor()
cursor.execute(update_query)
connection.commit()

In [36]:
query = "SELECT Name_of_School FROM dbo.Chicago WHERE Leaders_Score IS NULL"
data = pd.read_sql(query, connection)
data

,Name_of_School
